[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MobleyLab/drug-computing/blob/master/uci-pharmsci/assignments/MC/MC_assignment.ipynb)

# Monte Carlo assignment, PharmSci 175/275

Authors: David L. Mobley (UC Irvine) and M. Scott Shell (UCSB); adapted by Mobley from a similar assignment by Shell.

## Objective and intro:
In this assigment you will perform a basic Monte Carlo simulation for the same linear polymer system as in the Molecular Dynamics assignment (which is a prerequisite for this assignment), and compute thermodynamic properties for this system. 

### Overview:
Here, we will study the same simple model of polymers as in the MD assignment, where we have a series of Lennard-Jones spheres connected by harmonic spring “bonds”. Here, you will use Fortran routines that are provided for energy evaluations, and write your own Python Monte Carlo code for proposing and accepting or rejecting moves (as you will have seen in the Monte Carlo sandbox we tried in class). The Fortran code is provided, and you will write your own Python code. A template containing some functions (reused from the MD assignment) you may want is provided below. 
While this LJ system is, again, not of direct relevance to drug discovery, the Monte Carlo approach sees widespread use in a huge range of settings including molecular simulations but also in a wide range of other areas. It is very much worth being familiar with the basic approach.


## Your assignment

### Some settings
For the general background of what you’re simulating and the variables ($M$, $L$, $Cut$, etc...) see the [Molecular dynamics assignment](../MD/MD.ipynb). Again, the simulation will use periodic boundary conditions, a cutoff of $R_c = 2.5$, and no tail correction. Again, $N = 240$, $T = 1.0$, and $N/V = 0.8$ (dictating $L$). As usual in these assignments, we are using dimensionless variables. Since the system to be simulated is the same, this writeup will focus on details.

### Part A: Write a Python code for MC simulation of the polymeric system

Your task here is to write a Python code to perform MC simulations of the polymeric system. A Fortran library for evaluating energies of the system, given a position array, is provided (as in the last assignment). A template for your Python code is given below, along with documentation for the provided functions.

You will work in the canonical ensemble (NVT) here, and your MC moves should consist of single-atom displacements by random amounts in each of the x, y, and z directions. Each step, you will propose a move of some (randomly selected) atom by up to some maximum displacement in each of the x, y, and z directions. 

To make this process faster, do not evaluate the entire potential energy every time an atom is moved. Moving a single atom only changes interactions involving that atom, so you can compute only the interaction energy of that atom with other atoms. A Fortran function is provided that will do this in `mclib.f90` (which you will need to compile as usual via `f2py -c -m mclib mclib.f90`). This means that it is unnecessary to do a full energy evaluation for every proposed MC move. But note that, to get the energy change $\Delta U$, you need to calculate this interaction energy both before and after a proposed move. 


You can then keep a running total of the energy by doing updates of the form $U = U+\Delta U$. As noted in class, _only make this change after a move is accepted_. Also, for precision reasons, you will need to recompute the total energy occasionally (do this every 10 moves per particle (a move per particle just means you've done enough moves you could have moved each particle once, but since you pick particles randomly you won't necessarily do so), or 2,400 moves) by doing a full update of the system energy (with the provided Fortran function). 

During your MC simulation, you should compute the average acceptance ratio by keeping track of the number of proposed and the number of accepted moves. The acceptance ratio is the number accepted divided by the number proposed.

You will probably want to write a function `ProposeMove`: which takes a position array and a maximum step size; picks a random particle in the position array and displaces it by a distance up to the maximum step size in each direction, and returns the new proposed position array. You are free to write other functions also if they are helpful.

One practical consideration: Explicit copies of arrays are required to save positions before moving a particle. For example, you might do the following:

The `copy()` command is essential. If it is not used, `OldPos` will point to the same data in memory as `Pos` does, so when `Pos` is changed, `OldPos` will also change. To give an analogy, it’s like you and I are both working on the same shared document with our computers. If I change my copy, it will change the copy you see, since they are the same. To get around this, you must make a copy of the document after I tell you where it is, so that subsequent changes in my copy will not affect your copy.

_(Technical aside: The above code is not optimally efficient. The entire position array is copied to OldPos each time. In fact, it would probably be faster to copy only the position of the particle which will be moved, though this would require a more complicated version of ProposeMove)._

#### Fortran code provided

The `mclib` library provides:

## Installing Packages
***If you are running this on Google Colab, please add the installation blocks from the [getting started notebook](https://github.com/MobleyLab/drug-computing/blob/master/uci-pharmsci/Getting_Started.ipynb) or [condacolab](https://github.com/aakankschit/drug-computing/blob/master/uci-pharmsci/Getting_Started_condacolab.ipynb) here and then execute the code below***

#### Python functions provided

We provide an `MC_functions.py` python module which includes the same line search and conjugate gradient algorithms used previously, for your use here, as well as an `InitPositions(N, L)` function which will generate initial positions and return a position array.

#### Template Python code for Part A and following

In [ ]:
#import python modules
import numpy as np
import time
import pickle
import sys
import os

#import compiled fortran library
import mclib

# Import helper functions for line search, conjugate gradient as in MD assignment
from MC_functions import *

#========== GLOBAL VARIABLES ==========
#distance cutoff for pairwise interactions
Cut = 2.5
#set the random number seed; useful for debugging
#np.random.seed(342324)


#NOTE:
#You may assume unit masses in everything below 


# Write your `ProposeMove` function here, as well as perhaps code to test it and make sure it works.

ModuleNotFoundError: No module named 'mclib'

### Part B: Develop a run protocol for your code

For this, you may reuse any portions of the MD Assignment python code you find useful, incorporating them into your Python code for this assignment (some are already provided for you below). Your run protocol should:
- Initially place the atoms on a cubic lattice (as in the MD assignment)
- Energy minimize the initial configuration using conjugate-gradient
- Perform Monte Carlo steps

Perform simulations for M = 1, 2, and 4, with N = 240 as before. For each polymer length:
- Find the value of the position displacement step size that gives approx. 50% acceptance
- Perform 5 long trials, each starting with a different seed for the random number generator. You might try 500 steps per particle as a starting point.
- Average the average potential energy (averaged across the 5 trials) as a function of the number of MC steps; plot one series for each value of M
- Identify the "time" (in MC steps per particle) where these graphs suggest equilibration has occurred and the production period can begin. Indicate this point on your graph. (Note that this is the point where there is no longer a trend in energy)

Again, note that an "MC step per particle" does not mean that you necessarily have moved each particle once, but rather that you have taken one step for every particle in the system. So for example for a 10 particle system, one step per particle would correspond to 10 steps

Explain, for your writeup (which you can insert at the bottom of this document), how the optimum stepsize depends on the polymer chain length, and (if you can see any trend) how the equilibration time varies with chain length. Explain why.

In [ ]:
# Your work here

### Part C: Compute the average potential energy as a function of M

Perform additional simulations to compute $\left< U \right>$, the average potential energy, for each M. Use the equilibration times from above to discard some initial data for equilibration before collecting “production” data towards computing $\left< U \right>$.  That is, discard any data collected before the system is equilibrated. For each M, perform 5 trials and compute $\left< U \right>$ across all 5 trials and all MC steps. Collect data for at least 1000 steps per particle. (Note: You can use the same simulations for Part D below).

*_Extra credit_*: Compare your results to average energies from the MD Assignment. There may be some small differences. If you see any differences, explain their likely source (think ensembles). 

In [ ]:
# Your work here

### Part D: Compute the mean squared displacement
From the production period of Part C, and for each M, compute the mean-squared displacement as a function of MC steps per particle, averaged over the five trials. Graph these curves. If you still have the data, also plot the mean squared displacement from the MD Assignment, where the horizontal axis for the MD data should be number of MD steps and the MC data should have the number of MC steps per particle. This will compare how fast space is explored compared to the number of movements per particle. Which method explores phase space faster, MC or MD?  (To get good averages here, make sure you run long enough that each atom moves at least a couple of particle diameters over the course of the production run (on average)).

In [ ]:
# Your work here

## Do your report

### What to turn in

- Your Python code
- The step sizes resulting in approximately 50% acceptance and explain why they vary
- Average energies (Part C)
- Plots of the energy in equilibration (Part B) and of the mean squared displacement (Part D), compared with that from the MD assignment. For the purposes of these plots, you should roughly equate "MC steps" with "time" (though in fact they are not equivalent)
- The answer to the questions in Part B, and the extra credit in Part C if desired, and which method explores phase space faster (Part D).
- A brief summary of what you observed/learned (below) and your comments. Make sure I don't have to hunt for your answers to questions.
- Describe whether you think you got this right (why or why not)

These can be provided in this notebook; insertions above are fine. 

You may submit this notebook via the course website.


### Put your writeup here

Here, give your writeup/answer questions given above, discuss plots, summarize what you observed/learned, etc.

(Your writeup goes here)